In [2]:
import os

data_dir = "../dataset/augmented_resized_V2"
for split in ["train", "val", "test"]:
    split_dir = os.path.join(data_dir, split)
    print(f"{split} folder contains:")
    for class_dir in os.listdir(split_dir):
        class_path = os.path.join(split_dir, class_dir)
        print(f"  {class_dir}: {len(os.listdir(class_path))} images")


train folder contains:
  0: 55162 images
  1: 18470 images
  4: 9475 images
  3: 7936 images
  2: 24198 images
val folder contains:
  0: 6895 images
  1: 1840 images
  4: 1468 images
  3: 1000 images
  2: 3024 images
test folder contains:
  0: 6896 images
  1: 1862 images
  4: 1466 images
  3: 978 images
  2: 2999 images
